In [2]:
import pandas as pd
import shap
!pip install optuna
import optuna
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.7 MB/s eta 0:00:00


In [17]:
# Load the dataset
df = pd.read_csv("dropout.csv")

# Encode categorical variables (if any)
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Define features and target (assuming last column is the target)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Map target labels to class names
class_labels = {0: "graduate", 1: "dropout", 2: "enrolled"}
y = y.map(class_labels)

# Feature Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [18]:
# Ensure X is non-negative for chi2 (Modified #1 Preprocessing)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Feature Selection (Modified #1 - SelectKBest)
selector = SelectKBest(chi2, k=5)
X_selected = selector.fit_transform(X, y)

# Synthetic Oversampling (Modified #1 - Handling Class Imbalance)
smote = SMOTE(random_state=318945)
X_resampled, y_resampled = smote.fit_resample(X_selected, y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=318945)

# Polynomial Features (Modified #1 - Feature Engineering for Logistic Regression)
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

In [20]:
# Function to train, evaluate and return metrics
def train_and_evaluate(model, model_name, X_train, X_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    cm = confusion_matrix(y_test, y_pred, labels=["graduate", "dropout", "enrolled"])
    report = classification_report(y_test, y_pred, target_names=["graduate", "dropout", "enrolled"])

    print(f"\nModel: {model_name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(report)

    return model

# Baseline Models
rf_baseline = train_and_evaluate(RandomForestClassifier(random_state=318945), "Random Forest Baseline", X_train, X_test)
log_reg_baseline = train_and_evaluate(LogisticRegression(random_state=318945), "Logistic Regression Baseline", X_train, X_test)
svm_baseline = train_and_evaluate(SVC(kernel="linear", random_state=318945), "SVM Baseline", X_train, X_test)
nb_baseline = train_and_evaluate(GaussianNB(), "Naïve Bayes Baseline", X_train, X_test)

# Modified Algorithm #1 Models
rf_modified1 = train_and_evaluate(RandomForestClassifier(min_samples_split=5, class_weight="balanced", random_state=318945), "Random Forest Modified #1", X_train, X_test)
log_reg_modified1 = train_and_evaluate(LogisticRegression(C=0.5, max_iter=1000, random_state=318945), "Logistic Regression Modified #1", X_train_poly, X_test_poly)
svm_modified1 = train_and_evaluate(SVC(C=0.5, kernel="rbf", random_state=318945), "SVM Modified #1", X_train, X_test)
nb_modified1 = train_and_evaluate(GaussianNB(var_smoothing=1e-2), "Naïve Bayes Modified #1", X_train, X_test)

# Hyperparameter optimization with Optuna (Modified Algorithm #2)
def objective(trial):
    model_name = trial.suggest_categorical("model", ["Random Forest", "Logistic Regression", "SVM", "Naïve Bayes"])
    if model_name == "Random Forest":
        model = RandomForestClassifier(
            n_estimators=trial.suggest_int("n_estimators", 10, 200),
            max_depth=trial.suggest_int("max_depth", 3, 20),
            random_state=318945
        )
    elif model_name == "Logistic Regression":
        model = LogisticRegression(
            C=trial.suggest_loguniform("C", 1e-3, 1e3),
            max_iter=1000,
            random_state=318945
        )
    elif model_name == "SVM":
        model = SVC(
            C=trial.suggest_loguniform("C", 1e-3, 1e3),
            kernel="linear",
            random_state=318945
        )
    else:
        model = GaussianNB(var_smoothing=trial.suggest_loguniform("var_smoothing", 1e-9, 1e-1))

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred, average="weighted")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print("Best parameters:", study.best_params)

# Train and evaluate Optimized Models (Modified #2)
rf_optimized = RandomForestClassifier(n_estimators=study.best_params.get("n_estimators", 100), max_depth=study.best_params.get("max_depth", None), random_state=318945)
log_reg_optimized = LogisticRegression(C=study.best_params.get("C", 1.0), max_iter=1000, random_state=318945)
svm_optimized = SVC(C=study.best_params.get("C", 1.0), kernel="linear", random_state=318945)
nb_optimized = GaussianNB(var_smoothing=study.best_params.get("var_smoothing", 1e-9))

train_and_evaluate(rf_optimized, "Random Forest Optimized", X_train, X_test)
train_and_evaluate(log_reg_optimized, "Logistic Regression Optimized", X_train_poly, X_test_poly)
train_and_evaluate(svm_optimized, "SVM Optimized", X_train, X_test)
train_and_evaluate(nb_optimized, "Naïve Bayes Optimized", X_train, X_test)

# Dictionary to store accuracy scores
accuracy_scores = {}

# Baseline Models
accuracy_scores["Random Forest Baseline"] = train_and_evaluate(RandomForestClassifier(random_state=318945), "Random Forest Baseline", X_train, X_test)
accuracy_scores["Logistic Regression Baseline"] = train_and_evaluate(LogisticRegression(random_state=318945), "Logistic Regression Baseline", X_train, X_test)
accuracy_scores["SVM Baseline"] = train_and_evaluate(SVC(kernel="linear", random_state=318945), "SVM Baseline", X_train, X_test)
accuracy_scores["Naïve Bayes Baseline"] = train_and_evaluate(GaussianNB(), "Naïve Bayes Baseline", X_train, X_test)

# Modified Algorithm #1 Models
accuracy_scores["Random Forest Modified #1"] = train_and_evaluate(RandomForestClassifier(min_samples_split=5, class_weight="balanced", random_state=318945), "Random Forest Modified #1", X_train, X_test)
accuracy_scores["Logistic Regression Modified #1"] = train_and_evaluate(LogisticRegression(C=0.5, max_iter=1000, random_state=318945), "Logistic Regression Modified #1", X_train_poly, X_test_poly)
accuracy_scores["SVM Modified #1"] = train_and_evaluate(SVC(C=0.5, kernel="rbf", random_state=318945), "SVM Modified #1", X_train, X_test)
accuracy_scores["Naïve Bayes Modified #1"] = train_and_evaluate(GaussianNB(var_smoothing=1e-2), "Naïve Bayes Modified #1", X_train, X_test)

# Train and evaluate Optimized Models (Modified #2)
accuracy_scores["Random Forest Optimized"] = train_and_evaluate(rf_optimized, "Random Forest Optimized", X_train, X_test)
accuracy_scores["Logistic Regression Optimized"] = train_and_evaluate(log_reg_optimized, "Logistic Regression Optimized", X_train_poly, X_test_poly)
accuracy_scores["SVM Optimized"] = train_and_evaluate(svm_optimized, "SVM Optimized", X_train, X_test)
accuracy_scores["Naïve Bayes Optimized"] = train_and_evaluate(nb_optimized, "Naïve Bayes Optimized", X_train, X_test)


Model: Random Forest Baseline
Accuracy: 0.6923
F1 Score: 0.6916
Confusion Matrix:
[[335  75  48]
 [ 74 254  92]
 [ 47  72 329]]
Classification Report:
              precision    recall  f1-score   support

    graduate       0.63      0.60      0.62       420
     dropout       0.70      0.73      0.72       448
    enrolled       0.73      0.73      0.73       458

    accuracy                           0.69      1326
   macro avg       0.69      0.69      0.69      1326
weighted avg       0.69      0.69      0.69      1326


Model: Logistic Regression Baseline
Accuracy: 0.6320
F1 Score: 0.6376
Confusion Matrix:
[[266 142  50]
 [ 38 255 127]
 [ 10 121 317]]
Classification Report:
              precision    recall  f1-score   support

    graduate       0.49      0.61      0.54       420
     dropout       0.64      0.71      0.67       448
    enrolled       0.85      0.58      0.69       458

    accuracy                           0.63      1326
   macro avg       0.66      0.63    

[I 2025-02-20 02:56:33,011] A new study created in memory with name: no-name-b967e72c-819d-4810-9e21-612bc0de409e
<ipython-input-20-dc2053ec5bc1>:55: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  model = GaussianNB(var_smoothing=trial.suggest_loguniform("var_smoothing", 1e-9, 1e-1))
[I 2025-02-20 02:56:33,030] Trial 0 finished with value: 0.6231786518292287 and parameters: {'model': 'Naïve Bayes', 'var_smoothing': 8.488272602545059e-05}. Best is trial 0 with value: 0.6231786518292287.
<ipython-input-20-dc2053ec5bc1>:55: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  model = GaussianNB(var_smoothing=trial.suggest_loguniform("var_smoothing", 1e-9, 1e-1))
[I 2025-02-20 02:5


Model: SVM Modified #1
Accuracy: 0.6418
F1 Score: 0.6432
Confusion Matrix:
[[276 132  50]
 [ 49 230 141]
 [ 16  87 345]]
Classification Report:
              precision    recall  f1-score   support

    graduate       0.51      0.55      0.53       420
     dropout       0.64      0.77      0.70       448
    enrolled       0.81      0.60      0.69       458

    accuracy                           0.64      1326
   macro avg       0.66      0.64      0.64      1326
weighted avg       0.66      0.64      0.64      1326


Model: Naïve Bayes Modified #1
Accuracy: 0.6214
F1 Score: 0.6145
Confusion Matrix:
[[277 105  76]
 [ 55 174 191]
 [ 16  59 373]]
Classification Report:
              precision    recall  f1-score   support

    graduate       0.51      0.41      0.46       420
     dropout       0.58      0.83      0.69       448
    enrolled       0.80      0.60      0.69       458

    accuracy                           0.62      1326
   macro avg       0.63      0.62      0.61      

[I 2025-02-20 02:56:33,539] Trial 2 finished with value: 0.6889020948960678 and parameters: {'model': 'Random Forest', 'n_estimators': 83, 'max_depth': 19}. Best is trial 2 with value: 0.6889020948960678.
<ipython-input-20-dc2053ec5bc1>:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C=trial.suggest_loguniform("C", 1e-3, 1e3),
[I 2025-02-20 02:56:33,593] Trial 3 finished with value: 0.6363243470518782 and parameters: {'model': 'Logistic Regression', 'C': 501.19325468591944}. Best is trial 2 with value: 0.6889020948960678.
<ipython-input-20-dc2053ec5bc1>:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C=trial.suggest_loguniform("C", 1e-3, 1e3),
[I 2025-02-20 02:56:33

Best parameters: {'model': 'Random Forest', 'n_estimators': 172, 'max_depth': 12}

Model: Random Forest Optimized
Accuracy: 0.7157
F1 Score: 0.7173
Confusion Matrix:
[[326  85  47]
 [ 49 289  82]
 [ 28  86 334]]
Classification Report:
              precision    recall  f1-score   support

    graduate       0.63      0.69      0.66       420
     dropout       0.72      0.75      0.73       448
    enrolled       0.81      0.71      0.76       458

    accuracy                           0.72      1326
   macro avg       0.72      0.72      0.72      1326
weighted avg       0.72      0.72      0.72      1326


Model: Logistic Regression Optimized
Accuracy: 0.6463
F1 Score: 0.6492
Confusion Matrix:
[[273 135  50]
 [ 53 249 118]
 [ 12 101 335]]
Classification Report:
              precision    recall  f1-score   support

    graduate       0.51      0.59      0.55       420
     dropout       0.67      0.75      0.70       448
    enrolled       0.81      0.60      0.69       458

    acc